<a href="https://colab.research.google.com/github/ChetanB1997/ML_CardioVascular_risk_prediction/blob/main/cardiovascular_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Cardiovascular risk prediction



# **Project Summary -**

Heart disease, also known as cardiovascular disease, is a general term that refers to a group of conditions that affect the heart and blood vessels. The most common forms of heart disease include coronary artery disease, heart valve disease, and heart failure.

Current practices for the prevention and treatment of heart disease include lifestyle changes such as maintaining a healthy diet, regular exercise, and not smoking. Medications such as statins, blood pressure medications, and anticoagulants are also commonly used to lower the risk of heart disease. In some cases, interventional procedures such as angioplasty or bypass surgery may be necessary to treat advanced cases of heart disease.

Background research on heart disease has led to a better understanding of the underlying causes and risk factors for the condition. Some of the major risk factors for heart disease include high blood pressure, high cholesterol, diabetes, smoking, and a family history of the condition. Studies have also shown that certain lifestyle factors, such as diet and physical activity, can have a significant impact on the development of heart disease.

Recent research has also focused on identifying new treatments for heart disease, as well as ways to improve the effectiveness of existing treatments. This has led to the development of new medications and procedures, such as minimally invasive surgeries and the use of stem cells to repair damaged heart tissue.

Overall, the field of heart disease research is constantly evolving, with new findings and treatments being developed all the time

# **Problem Statement**


Do classification (Diseased or Not Diseased) on this data and choose the best model with highest accuracy.

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.feature_selection import chi2
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from prettytable import PrettyTable

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set()

### Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
path = '/content/drive/MyDrive/AlmaBetter/ML capstone /data_cardiovascular_risk.csv'

df = pd.read_csv(path , index_col = "id")

### Dataset First View

In [ ]:
# Dataset First Look
df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df.shape

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
df.duplicated().value_counts()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.isnull().sum()

In [ ]:
df.isna().sum().sum()

In [ ]:
# Plotting the heatmap for missing values
missing_values = df.isnull()
sns.heatmap(missing_values)

### What did you know about your dataset?

The dataset contains 3390 rows and 16 columns. There are no duplicate records, and contains total of 510 missing values.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df.columns

In [ ]:
# Dataset Describe
df.describe()

### Variables Description

**Sex:** male or female ("M" or "F")

**Age:** Age of the patient (Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)

**Education:** The level of education of the patient (categorical values - 1,2,3,4)

**is_smoking:** whether or not the patient is a current smoker ("YES" or "NO")

**Cigs Per Day:** the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

**BP Meds:** whether or not the patient was on blood pressure medication (Nominal)

**Prevalent Stroke:** whether or not the patient had previously had a stroke (Nominal)

**Prevalent Hyp:** whether or not the patient was hypertensive (Nominal)

**Diabetes:** whether or not the patient had diabetes (Nominal)

**Tot Chol:** total cholesterol level (Continuous)

**Sys BP:** systolic blood pressure (Continuous)

**Dia BP:** diastolic blood pressure (Continuous)

**BMI:** Body Mass Index (Continuous)

**Heart Rate:** heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)

**Glucose:** glucose level (Continuous)

**TenYearCHD:**10-year risk of coronary heart disease CHD(binary: “1”, means “Yes”, “0” means “No”)

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
df.nunique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Renaming the columns
df.rename(columns={'cigsPerDay':'cigs_per_day','BPMeds':'bp_meds',
                   'prevalentStroke':'prevalent_stroke','prevalentHyp':'prevalent_hyp',
                   'totChol':'total_cholesterol','sysBP':'systolic_bp','diaBP':'diastolic_bp',
                   'BMI':'bmi','heartRate':'heart_rate','TenYearCHD':'ten_year_chd'},
          inplace = True)


In [ ]:
# Defining 3 lists containing the column names of
# a. dependent variables
# b. continuous independent variables
# c. categorical independent variables
# This is defined based on the number of unique values for each attribute

dependent_var = ['ten_year_chd']
continuous_var = ['age','cigs_per_day','total_cholesterol','systolic_bp', 'diastolic_bp', 'bmi', 'heart_rate', 'glucose']
categorical_var = ['education', 'sex', 'is_smoking','bp_meds','prevalent_stroke', 'prevalent_hyp', 'diabetes']

In [ ]:
# Replacing the missing values in the categorical columns with its mode
df['education'] = df['education'].fillna(df['education'].mode()[0])
df['bp_meds'] = df['bp_meds'].fillna(df['bp_meds'].mode()[0])

In [ ]:
# education distribution after mode imputation
df.education.value_counts()

In [ ]:
# bp_meds distribution after mode imputation
df.bp_meds.value_counts()

###Variable: cigs_per_day

In [ ]:
# Mean and median number of cigarettes per day
print(df.cigs_per_day.mean().round(0))
print(df.cigs_per_day.median())

In [ ]:
df[df['cigs_per_day'].isna()]


here we see fo null values of cigs_per_day column are yes for every person who is smoking.means these all persons are smoker

In [ ]:

# mean and median number of cigarettes per day for a smoker (excluding non-smokers)
a=(df[df['is_smoking']=='YES']['cigs_per_day'].mean())
b=(df[df['is_smoking']=='YES']['cigs_per_day'].median())
print("Mean number of cigarettes for a smoker :",a)
print("Median number of cigarettes for a smoker :",b )

In [ ]:
# distribution of number of cigarettes per day for smokers (excluding non-smokers)
plt.figure(figsize=(8,5))
sns.distplot(df[df['is_smoking']=="YES"]['cigs_per_day'])
plt.axvline(df[df['is_smoking']=="YES"]['cigs_per_day'].mean(), color='orange', linestyle='dashed', linewidth=2)
plt.axvline(df[df['is_smoking']=="YES"]['cigs_per_day'].median(), color='red', linestyle='dashed', linewidth=2)
plt.title('Cigs_per_day for smokers distribution')
plt.show()

In [ ]:
# box plot for the number of cigarettes per day for smokers (excluding non-smokers)
plt.figure(figsize=(5,5))
sns.boxplot(df[df['is_smoking']=="YES"]['cigs_per_day'])

As the number of cigarettes smoked by the patients who smoke, contains some outliers so we cant use mean to replace the missing values. the missing values in ths cigs_per_day column can be replaced with its median value.

In [ ]:
df['cigs_per_day'] = df['cigs_per_day'].fillna(df[df['is_smoking']=="YES"]['cigs_per_day'].median())

In [ ]:
#converting this column into label (1,0)
df['is_smoking'] = np.where(df['is_smoking'] == 'YES',1,0)

###Variable:sex

In [ ]:
#Label Encoding Categorical columns sex and is_smoking
df['sex'] = df['sex'].apply(lambda x: 1 if x=='M' else 0)


###Variable: total_cholestrol, bmi, heart_rate

In [ ]:

variables = ['total_cholesterol', 'bmi', 'heart_rate']
num_plots = len(variables)

plt.figure(figsize=(12, 5))  # Adjust the figure size as needed

for idx, i in enumerate(variables):
    plt.subplot(1, num_plots, idx + 1)
    sns.distplot(df[i])
    plt.axvline(df[i].mean(), color='orange', linestyle='dashed', linewidth=2)
    plt.axvline(df[i].median(), color='red', linestyle='dashed', linewidth=2)
    plt.title(i + ' distribution')

plt.tight_layout()  # To ensure proper spacing between subplots
plt.show()

we see from graph all 3 column data is Positively skewed.

In [ ]:

# Checking outliers in total_cholesterol, bmi, heart_rate columns
var2=['total_cholesterol','bmi','heart_rate']
num_plots = len(var2)

plt.figure(figsize=(12, 5))

for idx, i in enumerate(var2):
  plt.subplot(1, num_plots, idx + 1)
  sns.boxplot(df[i])
  plt.title(i+' boxplot')

plt.tight_layout()  # To ensure proper spacing between subplots
plt.show()

All 3 columns contain the outliers

In [ ]:
# Mean and median for total_cholesterol
df.total_cholesterol.mean(),df.total_cholesterol.median()

In [ ]:
df.bmi.mean(), df.bmi.median()

In [ ]:
df.heart_rate.mean(), df.heart_rate.median()

Here we see the data is positively skewed as well it has many outliers so it will be good to use median for all missing values from this tables.

In [ ]:
# Imputing missing values in the total_cholesterol, bmi, and heart_rate with their medain values
df['total_cholesterol'] = df['total_cholesterol'].fillna(df['total_cholesterol'].median())
df['bmi'] = df['bmi'].fillna(df['bmi'].median())
df['heart_rate'] = df['heart_rate'].fillna(df['heart_rate'].median())

###Variable: Glucose

In [ ]:
df.glucose.isna().sum()

In [ ]:
# distribution of glucose
plt.figure(figsize=(8,5))
sns.distplot(df['glucose'])
plt.axvline(df['glucose'].mean(), color='orange', linestyle='dashed', linewidth=2)
plt.axvline(df['glucose'].median(), color='red', linestyle='dashed', linewidth=2)
plt.title('Glucose distribution')
plt.show()

The glucose is positively skewed

In [ ]:

# Outliers in glucose
plt.figure(figsize=(5,5))
sns.boxplot(df['glucose'])
plt.title('Glucose boxplot')

glucose has many outliers.

In [ ]:
df.glucose.mean(),df.glucose.median(),df.glucose.mode()

-Here 304 nearly 9 to 10 % values of data are missing.

-if we replace those values with mean /median it may create biased data.

-To avoid this we can impute the missing values using KNN imputer.

In [ ]:
# Using KNN imputer with K=10
imputer = KNNImputer(n_neighbors=10)
imputed = imputer.fit_transform(df)
df = pd.DataFrame(imputed, columns=df.columns)

In [ ]:
df.info()


###What all manipulations have you done and insights you found?

For categorical variable: replaced the null values with mode values

For Continuos variable:

-Total_cholestrol, BMI, hearRate: replaced the null values with median values

-cigsPerDay : replaced the null values with median value of people who smoke

-glucose : To avoid bias due to high number of null values, replaced null values with KNN Imputer.

In [ ]:
# changing datatypes
df = df.astype({'age': int, 'education':int,'sex':int,'is_smoking':int,'cigs_per_day':int,
               'bp_meds':int,'prevalent_stroke':int,'prevalent_hyp':int,'diabetes':int,
               'total_cholesterol':float,'systolic_bp':float,'diastolic_bp':float,
               'bmi':float,'heart_rate':float,'glucose':float,'ten_year_chd':int})

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
my_palette = {0 : 'orange' , 1 : 'teal'}
plt.figure(figsize = (8,8))
sns.countplot(x = df['diabetes'], hue = df['ten_year_chd'], palette = my_palette)
plt.title("Are diabetic patients at more risk of coronary heart disease?")
plt.legend(['No Risk','At Risk'])
plt.show()

##### 1. Why did you pick the specific chart?

The diebetic petients are at risk to cardiovascular risk.

##### 2. What is/are the insight(s) found from the chart?

yes, here diebetic patient count is very less as compaired to normal but diebetic patients are having 50 % chances of getting the risk.

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
my_palette = {0 : 'orange' , 1 : 'teal'}
plt.figure(figsize = (8,8))
sns.countplot(x = df['is_smoking'], hue = df['ten_year_chd'], palette = my_palette)
plt.title("Are smokers at more risk of CHD ?")
plt.legend(['No Risk','At Risk'])
plt.show()

##### 1. Why did you pick the specific chart?

to check if smocking affect the cardiovascular risk

##### 2. What is/are the insight(s) found from the chart?

number of smokers at risk are slightly more than non smokers at risk of CHD

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
plt.figure(figsize=(14,8))
ax=sns.countplot(x='age', hue='ten_year_chd', data=df)
ax.set_xlabel( "AgeCategory")
ax.set_ylabel( "AgeCategory Count")
ax.set_title(  "AgeCategory wise HeartDisease Tendency")
for i in ax.containers:
  ax.bar_label(i)
plt.show()

Here we see the age wise count of peoples having CHD. the age between 51-63 having more risk of CHD

#### Chart - 4

In [ ]:
import seaborn as sns

# Assuming you have imported the required libraries and loaded your DataFrame 'df'
# Also, make sure you have the list of categorical variables as 'categorical_var'

num_plots = len(categorical_var)
num_rows = 4
num_cols = 2

fig, axs = plt.subplots(num_rows, num_cols, figsize=(12,15))

# Add space between each plot
plt.subplots_adjust(hspace=0.5, wspace=0.3)

for idx, i in enumerate(categorical_var):
    row = idx // num_cols
    col = idx % num_cols
    p = sns.countplot(data=df, x=i, ax=axs[row, col])
    axs[row, col].set_xlabel(i)
    axs[row, col].set_title(i + ' distribution'+ '\n')
    for patch in p.patches:
        p.annotate(f'{patch.get_height()}', (patch.get_x() + patch.get_width() / 2., patch.get_height()), ha='center', va='center', xytext=(0,5), textcoords='offset points')

# To remove any extra empty subplots if there are fewer categorical variables than the grid size
for i in range(len(categorical_var), num_rows * num_cols):
    fig.delaxes(axs.flatten()[i])


Summary:

Majority of the patients belong to the education level 1, followed by 2, 3, and 4 respectively.

There are more female patients compared to male patients.

Almost half the patients are smokers.

100 patients under the study are undertaking blood pressure medication.

22 patients under the study have experienced a stroke.

1069 patients have hypertension.

87 patients have diabetes.

#### Chart - 5

In [ ]:
# Chart - 5 visualization code

plt.figure(figsize=(8, 8))
sns.boxplot(x='ten_year_chd', y='systolic_bp', data=df, palette=my_palette)
plt.title("Are patients with systolic BP at risk of CHD?")
plt.show()

#### Chart - 6

In [ ]:
# Chart - 6 visualization code
plt.figure(figsize = (8,8))
sns.boxplot(x='ten_year_chd',y='diastolic_bp', data=df, palette = my_palette)
plt.title("Are patients with Diastolic BP at risk of CHD?")
plt.show()

#### Chart - 7

In [ ]:
# Chart - 7 visualization code
for i in ['total_cholesterol','bmi','heart_rate']:
  plt.figure(figsize=(6,4))
  sns.distplot(df[i])
  plt.axvline(df[i].mean(), color='magenta', linestyle='dashed', linewidth=2)
  plt.axvline(df[i].median(), color='cyan', linestyle='dashed', linewidth=2)
  plt.title(i+' distribution')
  plt.show()

#### Chart - 8

In [ ]:
# Chart - 8 visualization code
def heartRate_data(row):
    if row['heart_rate'] <= 59:
        value = 'Low'
    elif row['heart_rate'] < 100:
        value = 'Normal'
    else:
        value = "High"

    return value

In [ ]:

df['heartRateLabel'] = df.apply(heartRate_data, axis = 1)

In [ ]:
df['heartRateLabel'].value_counts

In [ ]:
plt.figure(figsize = (8,8))
my_palette = {0 : 'orange' , 1 : 'teal'}
sns.countplot(x = df['heartRateLabel'], hue = df['ten_year_chd'], palette = my_palette)
plt.title("Is Heart rate is responsible for CHD ?? ")
plt.legend(['No Risk','At Risk'])
plt.show()


#### Chart - 9

In [ ]:
# Chart - 9 visualization code
plt.figure(figsize = (8,8))
my_palette = {0 : 'orange' , 1 : 'teal'}
sns.countplot(x = df['prevalent_stroke'], hue = df['ten_year_chd'], palette = my_palette)
plt.title("Whether a person who had a stroke earlier more prone to CHD?? ")
plt.legend(['No Risk' , 'At Risk'])
plt.show()


#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

# Updated correlations
plt.figure(figsize=(12,6))
plt.title('Correlation Analysis')
correlation = df[continuous_var].corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')


## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

###handling muticoliniarity


The variables systolic BP and diastolic BP are highly correlated.

In [ ]:
print(df['systolic_bp'].min(),df['systolic_bp'].max())
print(df['diastolic_bp'].min(),df['diastolic_bp'].max())

Pulse Pressure = Systolic BP - Diastolic BP

In [ ]:

# Creating a new column pulse_pressure
# and dropping systolic_bp and diastolic_bp

df['pulse_pressure'] = df['systolic_bp']-df['diastolic_bp']
df.drop('systolic_bp',axis=1,inplace=True)
df.drop('diastolic_bp',axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
continuous_var.remove('systolic_bp')
continuous_var.remove('diastolic_bp')
continuous_var.append('pulse_pressure')


In [ ]:
# Analyzing the distribution of pulse_pressure
plt.figure(figsize=(10,5))
sns.distplot(df['pulse_pressure'])
plt.axvline(df['pulse_pressure'].mean(), color='magenta', linestyle='dashed', linewidth=2)
plt.axvline(df['pulse_pressure'].median(), color='cyan', linestyle='dashed', linewidth=2)
plt.title('Pulse Pressure Distribution')


In [ ]:
# dropping is_smoking
df.drop('is_smoking',axis=1,inplace=True)

In [ ]:
df.drop(columns=['heartRateLabel'], inplace=True)

In [ ]:

# dropping is smoking
categorical_var.remove('is_smoking')
categorical_var

###outlier analysis


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(12,12))

# Flattening the axes for easier iteration
axes = axes.flatten()

# Looping through each continuous variable and plotting the boxplot
for i, col in enumerate(continuous_var):
    sns.boxplot(y=col, x=dependent_var[0], data=df, ax=axes[i])
    axes[i].set_title(col + ' boxplot')

# Tight layout to avoid overlapping titles and labels
plt.tight_layout()
plt.show()

here are outliers in the data, the effect of the outliers can be reduced to some extent by transforming it.
Once the data is transformed, if outliers beyond 3 standard deviations from the mean still remain, then they can be imputed with its respective median value.
This is done on the train data only to prevent data leakage.

In [ ]:
old_df= df.copy()

In [ ]:
# skewness along the index axis
(df[continuous_var]).skew(axis = 0)

In [ ]:
np.log10(df[continuous_var]+1).skew(axis = 0)

We can clearly see that by log transformation of the continuous variables, we are able to reduce it's skew to some extent.

In [ ]:
# Implementing log transformation on continuous variables

df['age']                   = np.log10(df['age']+1)
df['cigs_per_day']          = np.log10(df['cigs_per_day']+1)
df['total_cholesterol']     = np.log10(df['total_cholesterol']+1)
df['bmi']                   = np.log10(df['bmi']+1)
df['heart_rate']            = np.log10(df['heart_rate']+1)
df['glucose']               = np.log10(df['glucose']+1)
df['pulse_pressure']        = np.log10(df['pulse_pressure']+1)

In [ ]:
for col in continuous_var:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))  # Create a figure with two subplots side by side

    sns.distplot(old_df[col], ax=axes[0], label='old_df', color='blue')  # Plot on the first subplot
    sns.distplot(df[col], ax=axes[1], label='df', color='orange')        # Plot on the second subplot

    axes[0].axvline(old_df[col].mean(), color='magenta', linestyle='dashed', linewidth=2)  # Add mean line to the first subplot
    axes[0].axvline(old_df[col].median(), color='cyan', linestyle='dashed', linewidth=2) # Add median line to the first subplot

    axes[1].axvline(df[col].mean(), color='magenta', linestyle='dashed', linewidth=2)     # Add mean line to the second subplot
    axes[1].axvline(df[col].median(), color='cyan', linestyle='dashed', linewidth=2)      # Add median line to the second subplot

    axes[0].set_title(col + ' distribution (old_df)')  # Set title for the first subplot
    axes[1].set_title(col + ' distribution (df)')      # Set title for the second subplot

    axes[0].legend()  # Show legend for the first subplot
    axes[1].legend()  # Show legend for the second subplot

    plt.show()

Except cigs_per_day, we have successfully been able to reduce the skewness in the continuous variables. Now these distributions are closer to symmetric distribution.

##preprocessing


In [ ]:
# Defining dependent and independent variables
X = df.drop('ten_year_chd',axis=1)
y = df[dependent_var]

###Evalution metrices


Since the data we are dealing with is unbalanced, accuracy may not be the best evaluation metric to evaluate the model performance.

Also, since we are dealing with data related to healthcare, False Negatives are of higher concern than False Positive
In other words, it doesn’t matter whether we raise a false alarm but the actual positive cases should not go undetected

Considering these points in mind, it is decided that we use Recall as the model evaluation metric.

In [ ]:

# function to get recall score
def recall(actual,predicted):
  '''
  recall(actual,predicted)
  '''
  return recall_score(y_true=actual, y_pred=predicted, average='binary')

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y, shuffle=True)

In [ ]:
y_train.value_counts() , y_test.value_counts()

The train and test set contain almost equal proportion of results.

##### What data splitting ratio have you used and why?

Since the data is imbalanced, stratified split was employed to get almost equal proportion of dependent variables in the train and test sets.


### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Imputing the outliers in 'total_cholesterol', 'bmi', 'heart_rate', 'glucose', 'pulse_pressure' beyond 3 standard deviations from the mean with its median value.

In [ ]:
# Handling Imbalanced Dataset (If needed)
for i in ['total_cholesterol', 'bmi', 'heart_rate', 'glucose','pulse_pressure']:
  upper_lim = X_train[i].mean() + 3 * X_train[i].std()
  lower_lim = X_train[i].mean() - 3 * X_train[i].std()
  X_train.loc[(X_train[i] > upper_lim),i] = X_train[i].median()
  X_train.loc[(X_train[i] < lower_lim),i] = X_train[i].median()


In [ ]:
X_train[continuous_var].skew(axis = 0)

We were successsful in handling outliers in the train data, and thereby reducing skew.

###oversampling

In [ ]:
y_train.value_counts().plot(kind='bar', title='Target variable before SMOTE')

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Since we are dealing with unbalanced data, ie, only ~15% of the patients were diagnosed with coronary heart disease, we oversample the train dataset using SMOTE (Synthetic Minority Oversampling Technique).

In [ ]:
X_train

In [ ]:
# Oversampling using SMOTE
smote = SMOTE()

# fit predictor and target variable
X_smote, y_smote = smote.fit_resample(X_train, y_train)

print('Samples in the original dataset', len(y_train))
print('Samples in the resampled dataset', len(y_smote))


In [ ]:
# visualize the target variable after SMOTE
y_smote.value_counts().plot(kind='bar', title='Target variable after SMOTE')

We have successfully oversampled the minority class using SMOTE. Now the model we build will be able to learn from both the classes without any bias.

##scaling

Since the predictions from the distance based models will get affected if the attributes are in different ranges, we need to scale them.

We can use StandardScaler to scale down the variables.
The results obtained from scaling can be stored and used while building those models.

Tree algorithms do not necessarily require scaling.

In [ ]:
# Scaling data
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_smote_scaled = scaler.fit_transform(X_smote)
X_test_scaled = scaler.transform(X_test)

# Converting array to dataframe
X_train_scaled = pd.DataFrame(X_train_scaled,columns=X_train.columns)
X_smote_scaled = pd.DataFrame(X_smote_scaled,columns=X_smote.columns)
X_test_scaled = pd.DataFrame(X_test_scaled,columns=X_test.columns)

In [ ]:
# Scaled train values
X_train_scaled.head()

In [ ]:
# scaled SMOTE values
X_smote_scaled.head()

In [ ]:
# scaled test values
X_test_scaled.head()

## ***7. ML Model Implementation***

### ML Model - 1 :Linear Regression

In [ ]:
# ML Model - 1 Implementation
lr_model = LogisticRegression()

# Fit the Algorithm
lr_model.fit(X_smote_scaled, y_smote)

# Predict on the model
lr_train_pred = lr_model.predict(X_smote_scaled)

In [ ]:
# training set recall
lr_train_recall = recall(y_smote,lr_train_pred)
lr_train_recall

In [ ]:
# Test predictions
lr_test_pred = lr_model.predict(X_test_scaled)

# training set recall
lr_test_recall = recall(y_test,lr_test_pred)
lr_test_recall

In [ ]:
# Classification report
print(classification_report(y_test,lr_test_pred))

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import ConfusionMatrixDisplay as cmd
lr_confusion_matrix = cm(y_test, lr_test_pred)
cm_display = cmd(confusion_matrix = lr_confusion_matrix, display_labels = [False, True])

plt.figure(figsize=(4, 4))
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

cm_display.plot(cmap='Oranges')
plt.title('Confusion matrix: LOGISTIC REGRESSION')
plt.show()

False negatives obtained after using logistic regression: 50

### ML Model - 2 : K nearest neighbours

In [ ]:
# Value of k taken upto sqrt(n)
# Where n is no of records in the train dataset
# sqrt(4030) = 63.48
knn_test_res = []
knn_train_res = []
for k in range(1,65):
  knn_model = KNeighborsClassifier(n_neighbors=k)
  knn_model.fit(X_smote_scaled, y_smote)
  knn_train_pred = knn_model.predict(X_smote_scaled)
  knn_train_recall = recall(y_smote,knn_train_pred)
  knn_test_pred = knn_model.predict(X_test_scaled)
  knn_test_recall = recall(y_test,knn_test_pred)
  knn_test_res.append(knn_test_recall)
  knn_train_res.append(knn_train_recall)

In [ ]:
# Plotting the train and test recalls for different values of k
plt.figure(figsize=(10,5))
x_ = range(1,65)
y1 = knn_train_res
y2 = knn_test_res
plt.plot(x_, y1, label='Train Recall')
plt.plot(x_, y2, label = 'Test Recall')
plt.xlabel('K')
plt.ylabel('Recall')
plt.legend()
plt.show()

In [ ]:
best_k = knn_test_res.index(max(knn_test_res))+1
best_k

In [ ]:
# building knn model with best parameters
knn_model = KNeighborsClassifier(n_neighbors=best_k)


In [ ]:
# training the model
knn_model.fit(X_smote_scaled, y_smote)

In [ ]:
# Train predictions
knn_train_pred = knn_model.predict(X_smote_scaled)
# training set recall
knn_train_recall = recall(y_smote,knn_train_pred)
knn_train_recall

In [ ]:
# Test predictions
knn_test_pred = knn_model.predict(X_test_scaled)
# Test recall
knn_test_recall = recall(y_test,knn_test_pred)
knn_test_recall

In [ ]:
# Classification report
print(classification_report(y_test,knn_test_pred))

In [ ]:
# Confusion matrix
knn_confusion_matrix = cm(y_test, knn_test_pred)
cm_display = cmd(confusion_matrix = knn_confusion_matrix, display_labels = [False, True])

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

cm_display.plot(cmap='Oranges')
plt.title('Confusion matrix: K NEAREST NEIGHBORS')
plt.show()

### ML Model - 3 :Naive Bayes

In [ ]:
# ML Model - 3 Implementation
# Using stratified k fold cross validation so that each split
# has almost equal proportion of classification results
cv_method = RepeatedStratifiedKFold(n_splits=4,
                                    n_repeats=3,
                                    random_state=0)

# Fit the Algorithm
nb_model = GaussianNB()
nb_params = {'var_smoothing': np.logspace(0,-9, num=100)
             }
# Predict on the model

In [ ]:
nb_gridsearch = GridSearchCV(nb_model,
                             nb_params,
                             cv=cv_method,
                             scoring= 'recall')
nb_gridsearch.fit(X_smote_scaled,y_smote)
nb_best_params = nb_gridsearch.best_params_
nb_best_params

In [ ]:
# building knn model with best parameters
nb_model = GaussianNB(var_smoothing=nb_best_params['var_smoothing'])

In [ ]:
# training the model
nb_model.fit(X_smote_scaled, y_smote)

In [ ]:
# Train predictions
nb_train_pred = nb_model.predict(X_smote_scaled)
# training set recall
nb_train_recall = recall(y_smote,nb_train_pred)
nb_train_recall

In [ ]:
# Test predictions
nb_test_pred = nb_model.predict(X_test_scaled)
# Test recall
nb_test_recall = recall(y_test,nb_test_pred)
nb_test_recall


In [ ]:
# Classification report
print(classification_report(y_test,nb_test_pred))

In [ ]:
# Confusion matrix
nb_confusion_matrix = cm(y_test, nb_test_pred)
cm_display = cmd(confusion_matrix = nb_confusion_matrix, display_labels = [False, True])

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

cm_display.plot(cmap='Oranges')
plt.title('Confusion matrix: NAIVE BAYES')
plt.show()

False negatives obtained after using Naive Bayes: 72

##ML Model -4: Decision tree

In [ ]:
# Max depth of dt without hyperparameter tuning = 28 and min samples leaf = 1
dt_model = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10),
             'min_samples_split':np.arange(0.1,1,0.1),
             'min_samples_leaf':np.arange(0.1,0.6,0.1)
             }


In [ ]:
# using gridsearchcv to find best parameters
dt_gridsearch = GridSearchCV(dt_model,
                             dt_params,
                             cv=cv_method,
                             scoring= 'recall')
dt_gridsearch.fit(X_smote,y_smote)
dt_best_params = dt_gridsearch.best_params_
dt_best_params

In [ ]:
# building knn model with best parameters
dt_model = DecisionTreeClassifier(max_depth=dt_best_params['max_depth'],
                                  min_samples_split=dt_best_params['min_samples_split'],
                                  min_samples_leaf=dt_best_params['min_samples_leaf'])

dt_model.fit(X_smote_scaled, y_smote)

In [ ]:
# Train predictions
dt_train_pred = dt_model.predict(X_smote_scaled)
# training set recall
dt_train_recall = recall(y_smote,dt_train_pred)
dt_train_recall

In [ ]:
# Test predictions
dt_test_pred = dt_model.predict(X_test_scaled)
# Test recall
dt_test_recall = recall(y_test,dt_test_pred)
dt_test_recall

In [ ]:
# Classification report
print(classification_report(y_test,dt_test_pred))

In [ ]:

# Feature importances

dt_feat_imp = pd.Series(dt_model.feature_importances_, index=X.columns)
plt.figure(figsize=(10,5))
plt.title('Feature Importances: DECISION TREE')
plt.xlabel('Relative Importance')
dt_feat_imp.nlargest(20).plot(kind='barh')

Only one feature (age) was given imporatance while fitting a decision tree model. This is because of max depth being 1.



In [ ]:
# Confusion matrix
dt_confusion_matrix = cm(y_test, dt_test_pred)
cm_display = cmd(confusion_matrix = dt_confusion_matrix, display_labels = [False, True])

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

cm_display.plot(cmap='Oranges')
plt.title('Confusion matrix: DECISION TREE')
plt.show()

##ML Model - 5: Random Forest


In [ ]:

# random forest model
rf_model = RandomForestClassifier(random_state=0)
rf_params = {'n_estimators':[500],                    # limited due to computational power availability
             'max_depth':np.arange(1,6),
             'min_samples_split':np.arange(0.1,1,0.1),
             'min_samples_leaf':np.arange(0.1,0.6,0.1)}

In [ ]:
# using gridsearchcv to find best parameters
rf_gridsearch = GridSearchCV(rf_model,rf_params,cv=cv_method,scoring='recall')
rf_gridsearch.fit(X_smote,y_smote)
rf_best_params = rf_gridsearch.best_params_
rf_best_params

In [ ]:
# Fitting RF model with best parameters
rf_model = RandomForestClassifier(n_estimators=rf_best_params['n_estimators'],
                                  min_samples_leaf=rf_best_params['min_samples_leaf'],
                                  min_samples_split=rf_best_params['min_samples_split'],
                                  max_depth=rf_best_params['max_depth'],
                                  random_state=0)

# fit
rf_model.fit(X_smote,y_smote)

In [ ]:
# train predictions
rf_train_pred = rf_model.predict(X_smote)
# train recall
rf_train_recall = recall(y_smote,rf_train_pred)
rf_train_recall

In [ ]:
# Test predictions
rf_test_pred = rf_model.predict(X_test)
# test recall
rf_test_recall = recall(y_test,rf_test_pred)
rf_test_recall

In [ ]:
# Classification report
print(classification_report(y_test,rf_test_pred))

In [ ]:
# Feature importances

rf_feat_imp = pd.Series(rf_model.feature_importances_, index=X.columns)
plt.figure(figsize=(10,5))
plt.title('Feature Importances: RANDOM FORESTS')
plt.xlabel('Relative Importance')
rf_feat_imp.nlargest(20).plot(kind='barh')

Features other than age are also given importance in this case insted of just one feature in case of decision tree.

But age still remains the most important feature in predicting the final outcome for random forests.

In [ ]:
# Confusion matrix
rf_confusion_matrix = cm(y_test, rf_test_pred)
cm_display = cmd(confusion_matrix = rf_confusion_matrix, display_labels = [False, True])

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

cm_display.plot(cmap='Oranges')
plt.title('Confusion matrix: RANDOM FORESTS')
plt.show()

In [ ]:
# Plotting Recall scores

ML_models = ['Logistic Regression','K Nearest Neighbors','Naive Bayes','Decision Tree','Random Forests']
train_recalls = [lr_train_recall,knn_train_recall,nb_train_recall,dt_train_recall,rf_train_recall]
test_recalls = [lr_test_recall,knn_test_recall,nb_test_recall,dt_test_recall,rf_test_recall]

X_axis = np.arange(len(ML_models))

plt.figure(figsize=(10,5))
plt.barh(X_axis - 0.2, train_recalls, 0.4, label = 'Train Recall')
plt.barh(X_axis + 0.2, test_recalls, 0.4, label = 'Test Recall')

plt.yticks(X_axis,ML_models)
plt.xlabel("Recall score")
plt.title("Recall score for each model")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',title='Legend')
plt.show()


## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***